In [10]:
from __future__ import absolute_import
from __future__ import division

import tempfile
import urllib

import numpy as np
import tensorflow as tf

In [11]:
tf.logging.set_verbosity(tf.logging.INFO)
LEARNING_RATE = 0.001

In [12]:
def model_fn(features, targets, mode, params):
    #1. Configure the model via TF ops
    #2. Define the loss function for training/eval
    #3. Define the training operation/optimizer (i.e. to minimize the loss function)
    #4. Generate predictions
    
    first_hidden_layer = tf.contrib.layers.relu(features, 10)
    second_hidden_layer = tf.contrib.layers.relu(first_hidden_layer, 10)
    output_layer = tf.contrib.layers.linear(second_hidden_layer, 1)
    
    predictions = tf.reshape(output_layer, [-1])
    predictions_dict = {"age": predictions}
    
    #Calaculate the loss
    loss = tf.contrib.losses.mean_squared_error(predictions, targets)
    
    #Define the training op.
    train_op = tf.contrib.layers.optimize_loss(loss=loss,
                                               global_step=tf.contrib.framework.get_global_step(),
                                               learning_rate=params["learning_rate"],
                                               optimizer="SGD")
    
    return predictions, loss, train_op

In [31]:
def main(unused_argv):
    tr_set = tf.contrib.learn.datasets.base.load_csv_without_header(filename="dataset/abalone_train.csv", target_dtype=np.int, features_dtype=np.float64)
    ts_set = tf.contrib.learn.datasets.base.load_csv_without_header(filename="dataset/abalone_test.csv", target_dtype=np.int, features_dtype=np.float64)
    pr_set = tf.contrib.learn.datasets.base.load_csv_without_header(filename="dataset/abalone_predict.csv", target_dtype=np.int, features_dtype=np.float64)
    
    model_params = {"learning_rate": LEARNING_RATE}
    nn = tf.contrib.learn.Estimator(model_fn=model_fn, params=model_params)
    nn.fit(x=tr_set.data, y=tr_set.target, steps=100)
    
    ev = nn.evaluate(x=ts_set.data, y=ts_set.target, steps=1)
    loss_score = ev["loss"]
    print loss_score
    
    #Print out predictions
    predictions = nn.predict(x=pr_set.data, as_iterable=True)
    for i,p in enumerate(predictions): print i, p
    
    return 1
    

In [32]:
if __name__ == "__main__":
    tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_is_chief': True, 'save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe20246ce90>, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, 'tf_random_seed': None, 'keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', 'save_checkpoints_steps': None, '_master': '', 'keep_checkpoint_max': 5}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and bat

SystemExit: 1